In [ ]:
# IMPORT LIBRARIES

import os

# Exploratory data analysis (EDA) and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure plots appear in the notebook
%matplotlib inline

## Machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras import Model, Input 
from tensorflow.keras.layers import Dense 
from tensorflow.keras import Sequential
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import joblib

# Column transformations
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

## Model evaluation tools
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import RocCurveDisplay, roc_auc_score


# Print last updated timestamp
import time
print(f"Last updated: {time.asctime()}")

In [ ]:
# UCI heart disease dataset from Kaggle https://www.kaggle.com/datasets/sumaiyatasmeem/heart-disease-classification-dataset

# load file
df = pd.read_csv("../data/heart_disease_classification_dataset.csv")
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# df.shape
df

In [ ]:
# Number of positive (1) and negative (0) samples in our dataset
df.target.value_counts()

In [ ]:
# Normalized value counts
df.target.value_counts(normalize=True)

In [ ]:
# Number of missing values per column
missing_per_column = df.isna().sum()
print(missing_per_column)

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:

# CLEAN AND CONVERT DATA AND COLUMN TYPES

# Simple encoding of the categorical variables (applicable to variables with two values, like those below)
df['sex'] = df['sex'].map({'female':0, 'male':1})
df['target'] = df['target'].map({'no':0, 'yes':1})

# Initial handling of the missing values
df = df.replace({None: np.nan, '': np.nan, '?': np.nan})  # optional: convert None, blank or ? strings to NaN
df = df.fillna(np.nan)  # ensures any remaining missing values are np.nan

num_cols = df.select_dtypes(include=[np.number, 'int64', 'float64']).columns
print("Numeric columns before cleaning:", num_cols)
cat_cols = df.select_dtypes(include=['category', 'object']).columns
print("Categorical columns before cleaning:", cat_cols)
date_cols = list(df.select_dtypes(include=['datetime64']).columns)
print("Date columns before cleaning:", date_cols)


# # Replace '?' with NaN and convert to numeric (example)
# df['column'] = df['column'].replace('?', np.nan)
# df['column'] = pd.to_numeric(df['column'], errors='coerce')


# # CHECK DATA TYPES AFTER CLEANING (reports if any column has mixed types)
# for col in df.columns: 
#     types = df[col].map(type).unique() 
#     if len(types) > 1: 
#         print(col, types)


# print(df['thalach'].map(type).unique()) # see all data types in target column
# print(df['thalach'].unique())  # see all distinct values
# print(df['thalach'].dtype) # check data type of target column

# CONVERT SOME NUMERICAL COLUMNS TO CATEGORICAL (BASED ON THEIR VALUES)
df['sex'] = df['sex'].astype('category')
df['cp'] = df['cp'].astype('category')
df['fbs'] = df['fbs'].astype('category')
df['restecg'] = df['restecg'].astype('category')
df['exang'] = df['exang'].astype('category')
df['slope'] = df['slope'].astype('category')
df['ca'] = df['ca'].astype('category')
df['thal'] = df['thal'].astype('category')


# # CONVERT DATE-LIKE COLUMNS TO DATE COLUMNS
# for col in df.columns:
#     # Only check object or string columns
#     if df[col].dtype == 'object' or pd.api.types.is_string_dtype(df[col]):
#         parsed = pd.to_datetime(df[col], errors='coerce')
#         # If most of the column is valid dates, convert it
#         if parsed.notna().mean() > 0.8:  # adjust threshold if needed
#             df[col] = parsed



# #CONVERT NUMERIC COLUMNS WITH LOW UNIQUE VALUES TO CATEGORICAL
# num_cols = df.select_dtypes(include=[np.number, 'int64', 'float64']).columns
# print("Numeric columns:", num_cols)
# cat_cols = df.select_dtypes(include=['category', 'object']).columns
# print("Categorical columns:", cat_cols)
# numeric_like_categorical = []
# for col in num_cols:
#     print(col, df[col].nunique())
#     if df[col].nunique() <= 6:
#         numeric_like_categorical.append(col)
# print("Numeric-like categorical columns:", numeric_like_categorical)
# for col in numeric_like_categorical:
#     df[col] = df[col].astype('category')
#     # df[col] = df[col].cat.add_categories([-1]).fillna(-1)  # Handle missing values in categorical columns without Imputer
#     # df[col] = df[col].astype(str).fillna('missing').astype('category') # alternative way to handle missing values in categorical columns without Imputer


#PRINT COLUMNS BY DATA TYPE
num_cols = df.select_dtypes(include=[np.number, 'int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['category', 'object']).columns
date_cols = df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
print("Numeric columns after cleaning and conversion:", num_cols)
print("Categorical columns after cleaning and conversion:", cat_cols) 
print("Date columns:", date_cols)



In [ ]:
# PLOTTING DISTRIBUTION OF TARGET VALUES

#df.target.value_counts().plot(kind="bar", color=["salmon", "lightblue"]);
sns.barplot(x=df.target.value_counts().index, y=df.target.value_counts().values, palette=["salmon", "lightblue"]);
plt.xlabel("Target (0 = No Disease, 1 = Disease)")
plt.ylabel("Count")
plt.title("Distribution of Heart Disease in Dataset")
plt.show()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
# CROSSTAB TABLE - HEART DISEASE BY SEX

ct = pd.crosstab(df.target, df.sex)
ct

In [ ]:
# PLOTTING HEART DISEASE BY SEX

ct.plot(kind='bar')
plt.title('Heart Disease by Sex')
plt.xlabel('Sex (0=Female, 1=Male)')
plt.ylabel('Count')
plt.legend(['No Disease', 'Disease'])
plt.show()

# # Alternative visualization using seaborn
# sns.countplot(data=df, x='target', hue='sex')
# plt.xlabel('Heart Disease Status')
# plt.ylabel('Count')
# plt.title('Heart Disease by Sex')
# plt.legend(title='Sex', labels=['Female', 'Male'])
# plt.show()

In [ ]:
# PLOTTING AGE VS CHOLESTEROL BY HEART DISEASE STATUS

sns.scatterplot(x=df.age, y=df.chol, hue=df.target)
plt.title('Age vs Cholesterol by Heart Disease Status')
plt.xlabel('Age')
plt.ylabel('Cholesterol Level')
plt.legend(['No Disease', 'Disease'])
plt.show()

In [ ]:
# PLOTTING HEART DISEASE BY CHEST PAIN TYPE

sns.countplot(data=df, x='cp', hue='target')
plt.xlabel('Chest Pain Type')
plt.ylabel('Count')
plt.title('Heart Disease by Chest Pain Type')
plt.legend(title='Heart Disease Status', labels=['No Disease', 'Disease'])
plt.show()

In [ ]:
# CORRELATION HEATMAP FOR NUMERIC FEATURES

if len(num_cols) > 1:
    plt.figure(figsize=(10,6))
    sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
    plt.title('Numeric Feature Correlation')
    plt.show()

# Alternative
# corr_matrix = df.corr()
# plt.figure(figsize=(15, 10))
# sns.heatmap(corr_matrix, 
#             annot=True, 
#             linewidths=0.5, 
#             fmt= ".2f", 
#             cmap="YlGnBu");

In [ ]:
# REMOVE DUPLICATES

initial_count = df.shape[0]
df = df.drop_duplicates()
print(f"\nRemoved {initial_count - df.shape[0]} duplicate rows.")

In [ ]:
# # DETECT & REMOVE OUTLIERS

# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# from scipy.stats import zscore

# def remove_outliers(df, method='zscore', threshold=3, show_plots=False):
#     numeric_cols = df.select_dtypes(include=['number']).columns
#     if len(numeric_cols) == 0:
#         print("No numeric columns found.")
#         return df

#     print("Numeric columns for outlier detection:", list(numeric_cols))

#     # Optional boxplots
#     if show_plots:
#         for col in numeric_cols:
#             plt.figure()
#             sns.boxplot(x=df[col])
#             plt.title(f'Boxplot: {col}')
#             plt.show()

#     n_rows_before = len(df)  # store original number of rows

#     if method == 'zscore':
#         # Ensure output is always 2D NumPy array
#         z = np.abs(zscore(df[numeric_cols].to_numpy(), nan_policy='omit'))
#         outlier_mask = (z > threshold)
#         for i, col in enumerate(numeric_cols):
#             num_outliers = outlier_mask[:, i].sum()
#             print(f"{col}: {num_outliers} outliers ({num_outliers / n_rows_before * 100:.2f}%)")

#         # Keep only rows without any outlier
#         df = df[(z < threshold).all(axis=1)]

#     elif method == 'iqr':
#         for col in numeric_cols:
#             Q1 = df[col].quantile(0.25)
#             Q3 = df[col].quantile(0.75)
#             IQR = Q3 - Q1
#             mask = (df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)
#             num_outliers = (~mask).sum()
#             print(f"{col}: {num_outliers} outliers ({num_outliers / n_rows_before * 100:.2f}%)")
#             df = df[mask]

#     n_rows_after = len(df)
#     print(f"\nTotal rows removed: {n_rows_before - n_rows_after} ({(n_rows_before - n_rows_after) / n_rows_before * 100:.2f}%)")
#     print("Outliers handled.\n")
#     return df
# #df = remove_outliers(df, method='zscore', threshold=3, show_plots=False)

In [ ]:
# # DATE/TIME FEATURE EXTRACTION

# def extract_date_features(df, date_cols):
#     for col in date_cols:
#         df[col+'_year'] = df[col].dt.year
#         df[col+'_month'] = df[col].dt.month
#         df[col+'_day'] = df[col].dt.day
#         df[col+'_weekday'] = df[col].dt.weekday
#         df[col+'_week'] = df[col].dt.isocalendar().week
    
#     return df

In [ ]:
# # HANDLE SKEWNESS IN NUMERIC FEATURES

# def handle_skewness(df, threshold=0.75, method='yeo-johnson'):
#     numeric_cols = df.select_dtypes(include=np.number).columns
#     skewed_feats = df[numeric_cols].apply(lambda x: skew(x.dropna()))
#     skewed_feats = skewed_feats[abs(skewed_feats) > threshold].index
#     pt = PowerTransformer(method=method)
#     if len(skewed_feats) > 0:
#         df[skewed_feats] = pt.fit_transform(df[skewed_feats])
#         print(f"\nTransformed skewed numeric features: {list(skewed_feats)}")
#     else:
#         print("\nNo skewed numeric features detected.")
#     return df

In [ ]:
# # CORRELATION-BASED FEATURE SELECTION

# def remove_high_corr_features(df, threshold=0.9):
#     numeric_cols = df.select_dtypes(include=np.number).columns
#     corr_matrix = df[numeric_cols].corr().abs()
#     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#     to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
#     df = df.drop(columns=to_drop)
#     if to_drop:
#         print(f"\nRemoved highly correlated features: {to_drop}")
#     return df

In [ ]:
# # LOW VARIANCE FEATURE REMOVAL

# def remove_low_variance(df, threshold=0.01):
#     selector = VarianceThreshold(threshold)
#     numeric_cols = df.select_dtypes(include=np.number).columns
#     selector.fit(df[numeric_cols])
#     low_var_cols = numeric_cols[~selector.get_support()]
#     df = df.drop(columns=low_var_cols)
#     if len(low_var_cols) > 0:
#         print(f"\nRemoved low variance features: {list(low_var_cols)}")
#     return df

In [ ]:
np.random.seed(42)

In [ ]:
# DATA PREPARATION - CREATING FEATURES AND TARGET VARIABLE, CONVERTING TO NUMPY, GETTING NUMERICAL AND CATEGORICAL COLUMNS, AND THEIR INDICES

# Separate features and target variable
X = df.drop("target", axis=1).to_numpy()  # Feature matrix
y = df["target"].to_numpy()  # Select target variable and convert to np.array
#y = df.target.values # alternative way to convert to np.array

# Identify numeric and categorical columns and their indices
df = df.drop(columns=['target'], errors='ignore')
num_cols = df.select_dtypes(include=[np.number, 'int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['category', 'object']).columns
print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols) 
# Define column indices
num_cols_idx = df.columns.get_indexer(num_cols)
cat_cols_idx = df.columns.get_indexer(cat_cols)
print("Numeric column indices:", num_cols_idx)
print("Categorical column indices:", cat_cols_idx) 

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

In [ ]:
# SPLITTING INTO TRAINING AND TEST DATA, AND TRANSFORMING DATA

def split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx):
    """
    Splits the data into training and testing sets, applies imputation and encoding.
    
    Parameters:
    X : np.array
        Feature matrix.
    y : np.array
        Target vector.
    test_size : float
        Proportion of the dataset to include in the test split.
    num_cols_idx : list
        Indices of numeric columns
    cat_cols_idx : list
        Indices of categorical columns
    """

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Apply SimpleImputer to numeric columns
    imputer = SimpleImputer(strategy='mean')
    X_train[:,num_cols_idx] = imputer.fit_transform(X_train[:,num_cols_idx])
    X_test[:,num_cols_idx]  = imputer.transform(X_test[:,num_cols_idx])

    # Apply SimpleImputer to categorical columns
    imputer_cat = SimpleImputer(strategy='most_frequent')
    if len(cat_cols_idx) > 0:
        X_train[:,cat_cols_idx] = imputer_cat.fit_transform(X_train[:,cat_cols_idx])
        X_test[:,cat_cols_idx]  = imputer_cat.transform(X_test[:,cat_cols_idx])

    # Apply feature scaling to selected (true) numerical columns
    sc = StandardScaler()
    X_train[:,num_cols_idx] = sc.fit_transform(X_train[:,num_cols_idx])
    X_test[:,num_cols_idx] = sc.transform(X_test[:,num_cols_idx])

    # Apply OneHotEncoder to categorical columns
    ct = ColumnTransformer(
        transformers=[
            ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols_idx)
            ],
            remainder='passthrough'  # Keep non-specified columns as they are
        )
    

    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)

    # Creating LabelEncoder - optional
    le = LabelEncoder()
    # y_train = le.fit_transform(y_train)
    # y_test  = le.transform(y_test)
    return X_train, X_test, y_train, y_test, imputer, imputer_cat, sc, ct, le


In [ ]:
# # Detailed model evaluation for the Logistic Regression model (similar approach can be used for other models)

# # Identify numeric and categorical columns and their indices
# dfc = df.copy().drop(columns=['target'], errors='ignore')
# num_cols = dfc.select_dtypes(include=[np.number, 'int64', 'float64']).columns
# cat_cols = dfc.select_dtypes(include=['category', 'object']).columns
# print("Numeric columns:", num_cols)
# print("Categorical columns:", cat_cols) 
# # Define column indices
# num_cols_idx = dfc.columns.get_indexer(num_cols)
# cat_cols_idx = dfc.columns.get_indexer(cat_cols)
# print("Numeric column indices:", num_cols_idx)
# print("Categorical column indices:", cat_cols_idx) 

# # Separate features and target variable
# X = df.drop("target", axis=1).to_numpy()  # Feature matrix
# y = df["target"].to_numpy()  # Select target variable and convert to np.array
# #y = df.target.values # alternative way to convert to np.array

# print("Feature matrix shape:", X.shape)
# print("Target vector shape:", y.shape)

# def split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx):
#     """
#     Splits the data into training and testing sets, applies imputation and encoding.
    
#     Parameters:
#     X : np.array
#         Feature matrix.
#     y : np.array
#         Target vector.
#     test_size : float
#         Proportion of the dataset to include in the test split.
#     num_cols_idx : list
#         Indices of numeric columns
#     cat_cols_idx : list
#         Indices of categorical columns
#     """

#     # Split data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     # Apply SimpleImputer to numeric columns
#     imputer = SimpleImputer(strategy='mean')
#     X_train[:,num_cols_idx] = imputer.fit_transform(X_train[:,num_cols_idx])
#     X_test[:,num_cols_idx]  = imputer.transform(X_test[:,num_cols_idx])

#     # Apply SimpleImputer to categorical columns
#     imputer_cat = SimpleImputer(strategy='most_frequent')
#     if len(cat_cols_idx) > 0:
#         X_train[:,cat_cols_idx] = imputer_cat.fit_transform(X_train[:,cat_cols_idx])
#         X_test[:,cat_cols_idx]  = imputer_cat.transform(X_test[:,cat_cols_idx])

#     # Apply feature scaling to numerical columns
#     sc = StandardScaler()
#     X_train[:,num_cols_idx] = sc.fit_transform(X_train[:,num_cols_idx])
#     X_test[:,num_cols_idx] = sc.transform(X_test[:,num_cols_idx])

#     # Apply OneHotEncoder to categorical columns
#     ct = ColumnTransformer(
#         transformers=[
#             ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols_idx)
#             ],
#             remainder='passthrough'  # Keep non-specified columns as they are
#         )

#     X_train = ct.fit_transform(X_train)
#     X_test = ct.transform(X_test)

#     le = LabelEncoder()
#     y_train = le.fit_transform(y_train)
#     y_test  = le.transform(y_test)
#     return X_train, X_test, y_train, y_test, imputer, imputer_cat, sc, ct, le



# # Logistic Regression


# # No CV

# X_train, X_test, y_train, y_test, imputer, imputer_cat, sc, ct, le = split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx)


# classifier = LogisticRegression(max_iter=1000)

# classifier.fit(X_train, y_train)

# y_pred_train = classifier.predict(X_train)
# acc = accuracy_score(y_train, y_pred_train)
# #acc = classifier.score(X_train, y_train)
# print("Train Accuracy:", acc)


# y_pred_test = classifier.predict(X_test)
# acc = accuracy_score(y_test, y_pred_test)
# print("Test Accuracy:", acc)

# # With CV

# X_trainval, X_test, y_trainval, y_test, imputer, imputer_cat, sc, ct, le = split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx)

# classifier = LogisticRegression(max_iter=1000)

# scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro']
# results = cross_validate(classifier, X_trainval, y_trainval, cv=5, scoring=scoring)

# for metric in scoring:
#     print(f"CV Train {metric.capitalize()}:", results['test_' + metric])
#     print(f"CV Train {metric.capitalize()} mean:", results['test_' + metric].mean())
#     print(f"CV Train {metric.capitalize()} std:", results['test_' + metric].std())


# # ----- Final fit on full training data
# # This must be done because cross-validate function only provides metrics, not a trained model
# classifier.fit(X_trainval, y_trainval)

# # ----- Final evaluation on unseen test set -----
# y_pred_test = classifier.predict(X_test)


# acc = accuracy_score(y_test, y_pred_test)
# acc = classifier.score(X_test, y_test)
# # For binary classification
# # f1 = f1_score(y_test, y_pred_test)
# # For multiclass classification
# f1_macro = f1_score(y_test, y_pred_test, average='macro')  # average='micro' or 'weighted' also possible
# precision = precision_score(y_test, y_pred_test, average='macro')
# recall = recall_score(y_test, y_pred_test, average='macro')
# print("CV Test Accuracy:", acc)
# print("CV Test F1 Macro:", f1_macro)
# print("CV Test Precision Macro:", precision)
# print("CV Test Recall Macro:", recall)

In [ ]:
# COMPARING DIFFERENT CLASSIFIER MODELS

# X_train, X_test, y_train, y_test, imputer, imputer_cat, sc, ct, le = split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx)

# models = {"KNN": KNeighborsClassifier(),
#           "Logistic Regression": LogisticRegression(), 
#           "Random Forest": RandomForestClassifier(),
#           "XGBoost": XGBClassifier()}
 
# models_accuracy = {}
# # Loop through models
# for name, model in models.items():
#     # Fit the model to the data
#     model.fit(X_train, y_train)
#     # Evaluate the model and append its score to model_scores
#     models_accuracy[name] = model.score(X_test, y_test)

# models_accuracy

# Alternative approach using cross-validation (better since you dont want to use test data during model selection and tuning)

X_trainval, X_test, y_trainval, y_test, imputer, imputer_cat, sc, ct, le = split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx)


models = {
    "KNN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False)
}

models_accuracy = {}

# Loop through models
for name, model in models.items():
    # Perform 5-fold cross-validation on the training data
    cv_scores = cross_val_score(model, X_trainval, y_trainval, cv=5, scoring='accuracy')
    # Store the mean accuracy
    models_accuracy[name] = np.mean(cv_scores)

models_accuracy




In [ ]:
# CREATING DATAFRAME TO DISPLAY DATA ABOUT DIFFERENT MODELS AND THEIR ACCURACY

models_df = pd.DataFrame(models_accuracy, index=['accuracy'])
models_df = models_df.T.reset_index().rename(columns={'index': 'model'})
models_df

In [ ]:
# VISUALIZING DIFFERENT MODELS

sns.barplot(data=models_df, x='model', y='accuracy')

In [ ]:
# CREATING AND EVALUATING NEURAL NETWORK CLASSIFIER FOR HEART DISEASE PREDICTION


# model = Sequential([
#     Dense(10, activation='relu', input_shape=(13,)),
#     Dense(6, activation='relu'),
#     Dense(3, activation='relu'),
#     Dense(1, activation='sigmoid')  # binary output
# ])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

# model.fit(X_trainval, y_trainval, epochs=50, batch_size=16, validation_split=0.2)

# # Evaluate the model
# train_loss, train_accuracy = model.evaluate(X_trainval, y_trainval, verbose=0)
# print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")


# Alternative approach using Functional API
inputs = Input(shape=(30,)) 
x = Dense(10, activation='relu')(inputs) 
x = Dense(6, activation='relu')(x) 
x = Dense(3, activation='relu')(x) 
outputs = Dense(1, activation='sigmoid')(x) 
model = Model(inputs=inputs, outputs=outputs) 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
model.summary() 
 
model.fit(X_trainval, y_trainval, epochs=50, batch_size=16, validation_split=0.2)

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_trainval, y_trainval, verbose=0)
print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")


In [ ]:
import optuna

# HYPERPARAMETER TUNING FOR XGB ALGORITHM

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters to tune
    param = {
    'max_depth': trial.suggest_int('max_depth', 3, 10),
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
    'n_estimators': trial.suggest_int('n_estimators', 50, 500),
    'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    'gamma': trial.suggest_float('gamma', 0.0, 5.0),
    'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
    'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
    'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
    'eval_metric': 'logloss'
}

   
    # Create and train model
    model = XGBClassifier(**param)
    score = cross_val_score(model, X_trainval, y_trainval, cv=3, scoring='accuracy').mean()
    return score  # Optuna will try to maximize this

# Create a study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Creating final model with the best hyperparameters
best_model_xgb = XGBClassifier(**trial.params, eval_metric='logloss')


In [ ]:
# HYPERPARAMETER TUNING FOR LOGISTIC REGRESSION ALGORITHM

import optuna


X_trainval, X_test, y_trainval, y_test, imputer, imputer_cat, sc, ct, le = split_transform_data(X, y, test_size=0.2, num_cols_idx=num_cols_idx, cat_cols_idx=cat_cols_idx)

# Define the objective function for Optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial):
    # Define all valid (penalty, solver) combinations
    valid_combinations = [
        ('l1', 'liblinear'), 
        ('l1', 'saga'),
        ('l2', 'lbfgs'), 
        ('l2', 'liblinear'), 
        ('l2', 'sag'), 
        ('l2', 'saga'), 
        ('l2', 'newton-cg'),
        ('elasticnet', 'saga'),
        (None, 'lbfgs'), 
        (None, 'newton-cg'), 
        (None, 'sag'), 
        (None, 'saga')
    ]

    # Sample one valid combination
    penalty, solver = trial.suggest_categorical('combo', valid_combinations)

    # Only set l1_ratio if penalty is elasticnet
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0) if penalty == 'elasticnet' else None

    # Regularization strength
    C = trial.suggest_float('C', 0.001, 100.0, log=True)

    # Create Logistic Regression model
    model = LogisticRegression(
        penalty=penalty,
        solver=solver,
        C=C,
        l1_ratio=l1_ratio,
        max_iter=1000
    )

    # Evaluate using cross-validation
    score = cross_val_score(model, X_trainval, y_trainval, cv=5, scoring='accuracy').mean()
    return score


# Create study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Print best hyperparameters
trial = study.best_trial
print("Best trial:")
print(f"  Accuracy: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Extract penalty and solver from the 'combo' tuple
best_penalty, best_solver = trial.params['combo']
best_l1_ratio = trial.params.get('l1_ratio', None)
best_C = trial.params['C']

# Create final Logistic Regression model with the best hyperparameters
best_model_lr = LogisticRegression(
    penalty=best_penalty,
    solver=best_solver,
    C=best_C,
    l1_ratio=best_l1_ratio,
    max_iter=1000
)



In [ ]:
# EVALUATING AND COMPARING TWO BEST MODELS TO SELECT THE BETTER ONE

models = {
    "XGBoost": best_model_xgb,
    "Logistic Regression": best_model_lr
}

cv_results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_trainval, y_trainval, cv=5, scoring='accuracy')
    cv_results[name] = {
        "mean_accuracy": np.mean(scores),
        "std_accuracy": np.std(scores)
    }

for model_name, results in cv_results.items():
    print(f"{model_name}:")
    print(f"  Mean Accuracy: {results['mean_accuracy']:.4f}")
    print(f"  Std Accuracy: {results['std_accuracy']:.4f}")


In [ ]:
# TRAINING AND TESTING THE BEST SELECTED MODEL (WHICH IS LOGISTIC REGRESSION)

best_model_lr.fit(X_trainval, y_trainval)
y_pred_test = best_model_lr.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", acc)
f1_score = f1_score(y_test, y_pred_test)
print("Test F1 Score:", f1_score)
precision_score = precision_score(y_test, y_pred_test)
print("Test Precision Score:", precision_score)
recall_score = recall_score(y_test, y_pred_test)
print("Test Recall Score:", recall_score)
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print("Confusion Matrix:\n", confusion_matrix)


In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
sns.heatmap(confusion_matrix,annot=True, cbar=False)
plt.xlabel("true label")
plt.ylabel("predicted label")

In [ ]:
print(classification_report(y_test, y_pred_test))

In [ ]:
# PLOTTING THE METRICS OF THE BEST MODEL

metrics = pd.DataFrame({"Accuracy": acc,
                            "Precision": precision_score,
                            "Recall": recall_score,
                            "F1": f1_score},index=[0])

metrics

metrics.T.plot.bar(title = "Best (Logistic Regression) model metrics", legend=False);

In [ ]:
# PRINTING LEARNED COEFFICIENTS (WEIGHTS) FOR EACH FEATURE IN OUR BEST MODEL (LOGISTIC REGRESSION)
best_model_lr.coef_

In [ ]:
# Match features to columns
features_dict = dict(zip(df.columns, list(best_model_lr.coef_[0])))
features_dict

In [ ]:
# VISUALIZING FEATURE IMPORTANCE

features_df = pd.DataFrame(features_dict, index=[0])
features_df.T.plot.bar(title="Feature Importance", legend=False);

In [ ]:
# ROC CURVE FOR THE BEST MODEL

# from_estimator() = use a model to plot ROC curve on data

# Plotting the curve
RocCurveDisplay.from_estimator(estimator=best_model_lr, 
                               X=X_test, 
                               y=y_test); 

# Calculating AUC

y_pred_proba = best_model_lr.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC: {auc:.3f}")



In [ ]:
import shap


# Get feature names
num_feature_names = list(num_cols)
cat_feature_names = list(ct.named_transformers_['onehot'].get_feature_names_out(cat_cols))
feature_names = cat_feature_names + num_feature_names

# Convert to DataFrames
X_trainval_df = pd.DataFrame(X_trainval, columns=feature_names)
X_test_df = pd.DataFrame(X_test, columns=feature_names)

# # Check if those names are the same
# print(feature_names)
# print(X_trainval_df.columns)

# Create explainer
explainer = shap.LinearExplainer(best_model_lr, X_trainval_df)
shap_values = explainer.shap_values(X_test_df)

# SHAP summary plot
shap.summary_plot(shap_values, X_test_df, feature_names=feature_names)

# # SHAP force plot for one instance (individual explanation)
# shap.initjs()
# i = 0
# shap.force_plot(explainer.expected_value, shap_values[i], X_test_df.iloc[i])

# If you want explicit probability of 1 for this sample
# pred = model.predict_proba(X_test_df.iloc[[i]])[0,1]
# print(f"Predicted probability for sample {i}: {pred:.3f}")


# # If the plot is not clear, you can list all features and their SHAP values for this sample as a table
# sample_shap = pd.DataFrame({
#     'Feature': X_test_df.columns,
#     'Value': X_test_df.iloc[i].values,
#     'SHAP': shap_values[i]
# })
# # Add a column for absolute SHAP values
# sample_shap['Abs_SHAP'] = sample_shap['SHAP'].abs()
# # Sort by absolute contribution in descending order
# sample_shap_sorted = sample_shap.sort_values(by='Abs_SHAP', ascending=False).drop(columns='Abs_SHAP')

# sample_shap_sorted


SHAP Summary Plot

Y-axis represents features, and most important at the top.
Color denotes value of the feature (red = high value, blue = low value).

X-axis are SHAP value for that feature. Positive SHAP value for a feature means that feature pushes the prediction higher than the baseline, i.e., toward class 1. Negative SHAP value means that feature pushes the prediction lower than the baseline, i.e., toward class 0. SHAP shows how each feature “votes” to move the prediction away from the average.


Each dot represents one observation. Red dots are high value dots (for that feature), while blue dots are low value dots.
A dot far to the right → the feature pushes prediction higher than the baseline (toward positive class).
A dot far to the left → the feature pushes prediction lower than the baeline (toward negative class).

Feature importance is determined by the spread of SHAP values. The bigger the spread → the more influence that feature has.

SHAP Force Plot

Gives plot that contains the most important features, their values and visual representations of their SHAP values ie contributions to predictions (but it is better to see SHAP values in a table).



In [ ]:
from lime import lime_tabular

explainer = lime_tabular.LimeTabularExplainer(
    training_data=X_trainval,
    feature_names=feature_names,
    class_names=['No', 'Yes'],
    mode='classification'
)

# Explain a single prediction
i = 15 # row in X_test dataset
exp = explainer.explain_instance(
    data_row=X_test_df.iloc[i],
    predict_fn=model.predict_proba,
    num_features=10 # how many features to display
)

# Printing features and their LIME values, ie contributions
lime_exp_list = exp.as_list()
print(lime_exp_list)

exp.show_in_notebook(show_table=True)


LIME explains a single prediction by approximating the model locally with a simple interpretable model (like a linear regression) around that sample (example, instance). It assigns LIME values to features showing how much each one contributed to increasing or decreasing the prediction for that specific instance.

In the upper left corner LIME displays prediction probabilities for that specific instance.

Chart gives LIME values ie contributions for each feature (values 0.09, 0.06 etc), while color represents class (positive orange, negative blue). If LIME value is orange it means that value is positive, and thus contributing to positive (1) class. If LIME value is blue it means that value is negative, and thus contributing to negative (0) class. However, chart gives only absolute values, using color to denote contribution to class. Negative values can be seen using exp.as_list().

Table:
Feature column - the name of the feature (or one-hot encoded feature).
Value column - the value of the feature (normalized since we used scaling by StandardScaler).

In [ ]:
# SAVE MODEL COMPONENTS

joblib.dump(imputer, "../models/imputer.joblib")
joblib.dump(imputer_cat, "../models/imputer_cat.joblib")
joblib.dump(sc, "../models/sc.joblib")
joblib.dump(ct, "../models/ct.joblib")
joblib.dump(le, "../models/le.joblib")
joblib.dump(best_model_lr, "../models/model.joblib")

print("Saved: model, imputers, scaler, transformer, and label encoder.")